In [6]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.vectorstores.cassandra import Cassandra
import cassio
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
groq_api_key = os.environ['GROQ_API_KEY']
ASTRA_DB_APPLICATION_TOKEN="AstraCS:MoSHHhIxBPvcBlSZsDMDXCXM:7928de80ad1f6e930ddea1b068e8e3b899624271efbcd13d2e3d8870dccd6e99"
ASTRA_DB_ID="646e76a2-9946-4c52-b024-ef8926e4c3bc"
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [11]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")

                     )))

text_documents=loader.load()

In [42]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter= RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs=text_splitter.split_documents(text_documents)

In [44]:
from langchain.llms import Ollama
ollama = Ollama(base_url='http:localhost:4000', model='llama2')

In [45]:
embeddings = OllamaEmbeddings()
astra_vector_store = Cassandra(
    embedding=embeddings,
    table_name="web_based_loader",
    session=None,
    keyspace=None
)

In [46]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
astra_vector_store.add_documents(docs)
print("Inserted %i headlines." % len(docs))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 66 headlines.


In [47]:
llm=ChatGroq(groq_api_key=groq_api_key,
         model_name="mixtral-8x7b-32768")

from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>

Question: {input}""")

In [48]:
astra_vector_index.query("Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique",llm=llm)

"It seems like you're looking for information and guidance related to various resources and papers about language models, tool usage, and file naming conventions. I'll first address the goals, constraints, and resources you provided, and then I'll give a brief overview of the relevant papers and file naming conventions.\n\nGoals, constraints, and resources:\n1. Your goals are not explicitly stated, so I will provide general information that could be helpful.\n2. The constraints include a 4000-word limit, no user assistance, using specific commands, and using subprocesses for long-running commands.\n3. Resources include internet access, long-term memory management, GPT-3.5 agents for simple tasks, and file output.\n\nPapers and overviews:\n1. Schick et al. (2023) - Toolformer: This paper discusses language models teaching themselves to use tools. This could be relevant if your goal is to create a model that can use tools or APIs effectively.\n2. Li et al. (2023) - API-Bank: This paper i

In [49]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

retriever=astra_vector_store.as_retriever()
document_chain=create_stuff_documents_chain(llm,prompt)
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [50]:
response=retrieval_chain.invoke({"input":"Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique"})
response

{'input': 'Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique',
 'context': [Document(page_content='[13] Schick et al. “Toolformer: Language Models Can Teach Themselves to Use Tools.” arXiv preprint arXiv:2302.04761 (2023).\n[14] Weaviate Blog. Why is Vector Search so fast? Sep 13, 2022.\n[15] Li et al. “API-Bank: A Benchmark for Tool-Augmented LLMs” arXiv preprint arXiv:2304.08244 (2023).\n[16] Shen et al. “HuggingGPT: Solving AI Tasks with ChatGPT and its Friends in HuggingFace” arXiv preprint arXiv:2303.17580 (2023).\n[17] Bran et al. “ChemCrow: Augmenting large-language models with chemistry tools.” arXiv preprint arXiv:2304.05376 (2023).\n[18] Boiko et al. “Emergent autonomous scientific research capabilities of large language models.” arXiv preprint arXiv:2304.05332 (2023).\n[19] Joon Sung Park, et al. “Generative Agents: Interactive Simulacra of Human Behavior.” arXiv preprint arXiv:2304.03442 (2023).\n[20] AutoGPT. https://github.com/Significant-G

In [51]:
response["answer"]

"To determine if chain of thought (CoT) has become a standard prompting technique, I will first clarify what CoT is and then examine the given resources to see if it is mentioned or used.\n\nChain of thought (CoT) is a prompting technique where a language model is encouraged to generate a series of intermediate steps or thoughts before producing the final answer. This technique aims to improve the model's problem-solving abilities and provide more insight into its decision-making process. (Note: Wei et al. 2022 is not part of the provided resources, so I cannot directly reference their work.)\n\nNow, I will analyze the given resources:\n\n1. [13] Schick et al. “Toolformer: Language Models Can Teach Themselves to Use Tools.” (2023): This paper presents a method for teaching language models to use tools, but it does not mention or use CoT as a prompting technique.\n2. [14] Weaviate Blog. “Why is Vector Search so fast?” (2022): This blog post discusses vector search but does not mention o